#### DataSet Downloads

https://www.kaggle.com/datasets/rajathmc/cornell-moviedialog-corpus

#### Arquivos Usados
* movie_lines.txt
* movie_conversations.txt

In [1]:
import numpy as np
import tensorflow as tf
import re
import time
import pandas as pd
from deep_translator import GoogleTranslator

2023-06-23 22:42:56.466072: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-23 22:42:56.500283: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-23 22:42:56.501098: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-23 22:42:57.207484: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Pré Processamento da Base

In [2]:
path_root = '/opt/dna/chat_bot/datalake/transient/'
filename = 'movie_lines.txt'
linhas = open(path_root + filename, encoding='utf-8').read().split('\n')

In [3]:
linhas[:10]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?']

In [4]:
len(linhas)

304714

In [5]:
path_root = '/opt/dna/chat_bot/datalake/transient/'
filename = 'movie_conversations.txt'
conversas = open(path_root + filename, encoding='utf-8').read().split('\n')

In [6]:
conversas[:10]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']"]

In [7]:
len(conversas)

83098

In [8]:
id_para_linha = {}
for linha in linhas:
    _linha = linha.split(' +++$+++ ')
    if len(_linha) == 5:
        id_para_linha[_linha[0]] = _linha[4]

In [9]:
id_para_linha['L194']

'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.'

In [10]:
id_para_linha['L195']

"Well, I thought we'd start with pronunciation, if that's okay with you."

#### Criação das listas com todas as conversas

In [11]:
conversas_id = []
for conversa in conversas[:-1]:
    _conversa = conversa.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conversas_id.append(_conversa.split(','))

In [12]:
conversas_id[:10]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366']]

#### Separação das perguntas e respostas

In [13]:
perguntas = []
respostas = []
for conversa in conversas_id:
    #print(conversa)
    #print('****')
    for i in range(len(conversa) - 1):
        perguntas.append(id_para_linha[conversa[i]])
        respostas.append(id_para_linha[conversa[i + 1]])

In [14]:
perguntas[:10]

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
 "Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "You're asking me out.  That's so cute. What's your name again?",
 "No, no, it's my fault -- we didn't have a proper introduction ---",
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Why?',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 'Gosh, if only we could find Kat a boyfriend...']

In [15]:
respostas[:10]

["Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?",
 'Forget it.',
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Seems like she could get a date easy enough...',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 "That's a shame.",
 'Let me see what I can do.']

In [16]:
len(perguntas), len(respostas)

(221616, 221616)

#### Teste com tradutor

In [17]:
#to_translate = ' '.join(perguntas[:100])
#perguntas_transleted = GoogleTranslator(source='english', target='portuguese').translate(to_translate)
#perguntas_transleted

In [18]:
#to_translate = ' '.join(respostas[:100])
#respostas_transleted = GoogleTranslator(source='english', target='portuguese').translate(to_translate)
#respostas_transleted

### Limpeza de texto

In [19]:
def limpa_texto(texto):
    texto = texto.lower()
    texto = re.sub(r"i'm", "i am", texto)
    texto = re.sub(r"you're", "you are", texto)
    texto = re.sub(r"aren't", "are not", texto)
    texto = re.sub(r"he's", "he is", texto)
    texto = re.sub(r"she's", "she is", texto)
    texto = re.sub(r"that's", "that is", texto)
    texto = re.sub(r"what's", "what is", texto)
    texto = re.sub(r"where's", "where is", texto)
    texto = re.sub(r"don't", "do not", texto)
    texto = re.sub(r"didn't", "did not", texto)
    texto = re.sub(r"doesn't", "does not", texto)
    texto = re.sub(r"who's", "who is", texto)
    texto = re.sub(r"isn't", "not is", texto)
    texto = re.sub(r"let's", "let is", texto)
    texto = re.sub(r"mom's", "mother", texto)
    texto = re.sub(r"\'ll", " will", texto)
    texto = re.sub(r"\'ve", " have", texto)
    texto = re.sub(r"\'re", " are", texto)
    texto = re.sub(r"\'d", " would", texto)
    texto = re.sub(r"won't", "will not", texto)
    texto = re.sub(r"can't", "cannot", texto)
    texto = re.sub(r"c'mon", "came on", texto)
    
    texto = re.sub(r"[-()\"#/@;:<>{}`+=~|.?,]", "", texto)
    return texto

In [20]:
perguntas_limpas = []
for pergunta in perguntas:
    perguntas_limpas.append(limpa_texto(pergunta))

In [21]:
respostas_limpas = []
for resposta in respostas:
    respostas_limpas.append(limpa_texto(resposta))

In [22]:
perguntas_limpas[:5]

['can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again',
 'well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 'you are asking me out  that is so cute what is your name again',
 "no no it's my fault  we did not have a proper introduction "]

In [23]:
respostas_limpas[:5]

['well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 "okay then how 'bout we try out some french cuisine  saturday  night",
 'forget it',
 'cameron']

### Criação de um dicionário que mapeia cada palavra e o numero de ocorrencias

In [24]:
palavras_contagem = {}
for pergunta in perguntas_limpas:
    for palavra in pergunta.split():
        if palavra not in palavras_contagem:
            palavras_contagem[palavra] = 1
        else:
            palavras_contagem[palavra] += 1 
            
for resposta in respostas_limpas:
    for palavra in resposta.split():
        if palavra not in palavras_contagem:
            palavras_contagem[palavra] = 1
        else:
            palavras_contagem[palavra] += 1             

In [25]:
palavras_contagem

{'can': 15975,
 'we': 40500,
 'make': 6747,
 'this': 33573,
 'quick': 337,
 'roxanne': 1,
 'korrine': 1,
 'and': 65581,
 'andrew': 56,
 'barrett': 19,
 'are': 56024,
 'having': 1217,
 'an': 9482,
 'incredibly': 60,
 'horrendous': 4,
 'public': 364,
 'break': 895,
 'up': 16050,
 'on': 27769,
 'the': 140638,
 'quad': 2,
 'again': 3193,
 'well': 14090,
 'i': 204493,
 'thought': 4550,
 'would': 20005,
 'start': 1656,
 'with': 24954,
 'pronunciation': 2,
 'if': 18949,
 'that': 67068,
 'is': 87840,
 'okay': 6096,
 'you': 212914,
 'not': 84900,
 'hacking': 18,
 'gagging': 9,
 'spitting': 16,
 'part': 1419,
 'please': 3208,
 'asking': 746,
 'me': 44888,
 'out': 18466,
 'so': 19059,
 'cute': 272,
 'what': 55197,
 'your': 29938,
 'name': 3122,
 'no': 27572,
 "it's": 25843,
 'my': 29683,
 'fault': 482,
 'did': 21430,
 'have': 46591,
 'a': 101997,
 'proper': 138,
 'introduction': 19,
 'cameron': 35,
 'thing': 5731,
 'am': 37860,
 'at': 15290,
 'mercy': 68,
 'of': 56294,
 'particularly': 111,
 'hid